In [1]:
import glob
import math

import meteostat
import os
import pandas as pd
from datetime import datetime
from tqdm.auto import tqdm
import numpy as np

path = "transitpredictor/data"

translate_stop_ids = {
    70001: 0,
    70003: 1,
    70005: 2,
    70007: 3,
    70009: 4,
    70011: 5,
    70013: 6,
    70015: 7,
    70017: 8,
    70019: 9,
    70021: 10,
    70023: 11,
    70025: 12,
    70027: 13,
    70029: 14,
    70031: 15,
    70279: 16,
    70033: 17,
    70035: 18,
    70036: 19
}

translate_event_type = {
    "ARR": "ARR",
    "DEP": "DEP",
    "PRA": "ARR"
}

In [7]:
csv_filepath = path + "/Events_2019"
files = glob.iglob(os.path.join(csv_filepath, "*.csv"))

df = pd.concat((pd.read_csv(f) for i, f in enumerate(files) if i == 0))
df = df[(df.route_id == "Orange") & (df.direction_id == 1)]
df = df.drop(columns=["direction_id", "stop_sequence", "vehicle_id", "vehicle_label"])
df = df.sort_values(["service_date","trip_id", "event_time"])
df["stop_id"] = df["stop_id"].replace(translate_stop_ids)
df["event_type"] = df["event_type"].replace(translate_event_type)
df["event_time"] = df["event_time"].apply(datetime.fromtimestamp)

In [3]:
dfs_per_day = list(df.groupby("service_date"))
len(dfs_per_day[0][1])
df

,service_date,route_id,trip_id,stop_id,event_type,event_time,event_time_sec
34,2019-01-01,Orange,38939742,0,ARR,2019-01-02 00:12:33,87153
24,2019-01-01,Orange,38939742,0,DEP,2019-01-02 00:16:09,87369
7,2019-01-01,Orange,38939742,1,ARR,2019-01-02 00:17:43,87463
25,2019-01-01,Orange,38939742,1,DEP,2019-01-02 00:18:23,87503
26,2019-01-01,Orange,38939742,2,ARR,2019-01-02 00:19:22,87562
...,...,...,...,...,...,...,...
2701761,2019-03-31,Orange,ADDED-1553786854,16,DEP,2019-03-31 22:23:18,80598
2701749,2019-03-31,Orange,ADDED-1553786854,17,ARR,2019-03-31 22:24:18,80658
2701727,2019-03-31,Orange,ADDED-1553786854,17,DEP,2019-03-31 22:26:41,80801
2701740,2019-03-31,Orange,ADDED-1553786854,18,ARR,2019-03-31 22:30:12,81012


In [39]:
results = {}

dfs_per_day = list(df.groupby("service_date"))

for date, d in tqdm(dfs_per_day[:1]):
    events_by_train = list(t_events[(t_events["event_type"] == "ARR") | (t_events["event_type"] == "DEP")].drop_duplicates() for _, t_events in d.groupby("trip_id"))
    full_runs = list(t_events for t_events in events_by_train if len(t_events) == 39)
    if len(full_runs) == 0:
        print("Skipping " + date)
        continue
    full_runs = pd.concat(full_runs)
    tids, _ = pd.factorize(full_runs["trip_id"])
    full_runs["train_id"] = tids

    runs_minim = full_runs[["train_id", "stop_id", "event_time", "event_time_sec", "event_type"]].groupby("train_id")
    e = []
    for tid, r in runs_minim:
        arrs = r[r["event_type"] == "ARR"]
        deps = r[r["event_type"] == "DEP"]

        event_pairs = arrs.join(deps, how="cross", lsuffix="_arr", rsuffix="_dep")
        event_pairs = event_pairs[event_pairs["stop_id_arr"] > event_pairs["stop_id_dep"]] # Sequential order
        event_pairs["total_travel_time"] = event_pairs["event_time_sec_arr"] - event_pairs["event_time_sec_dep"]
        event_pairs = event_pairs[["train_id_arr", "stop_id_dep", "stop_id_arr", "total_travel_time", "event_time_arr", "event_time_sec_arr"]]
        event_pairs.rename(columns={"train_id_arr":"train_id"}, inplace=True)
        event_pairs = event_pairs.sort_values(["train_id", "stop_id_dep", "stop_id_arr"])
        e.append(event_pairs)

    runs = full_runs

    # Calculate dwell time
    runs[["prev_event_type", "prev_stop_id", "prev_trip_id", "prev_event_time_sec"]] = runs[["event_type", "stop_id", "trip_id", "event_time_sec"]].shift(1)
    runs["dwell_time"] = np.where((runs["prev_event_type"] == "ARR") & (runs["event_type"] == "DEP") & (runs["prev_stop_id"] == runs["stop_id"]) & (runs["prev_trip_id"] == runs["trip_id"]), runs["event_time_sec"] - runs["prev_event_time_sec"], math.nan)
    runs = runs.drop(columns=["prev_event_type", "prev_stop_id", "prev_trip_id", "prev_event_time_sec"])

    # We no longer care about arrivals at all
    runs = runs[runs["event_type"] == "DEP"]
    runs = runs.drop(columns=["event_type"])

    # Pair up each train for the day with the next one that departs from the same station
    runs = runs.sort_values(["stop_id", "event_time_sec"])
    runs[["prev_stop_id", "prev_event_time_sec"]] = runs[["stop_id", "event_time_sec"]].shift(1)
    runs["headway"] = np.where(runs["prev_stop_id"] == runs["stop_id"], runs["event_time_sec"] - runs["prev_event_time_sec"], math.nan)
    runs = runs.drop(columns=["prev_stop_id", "prev_event_time_sec"])

    forest_hills_deps = runs[runs["stop_id"] == 0].drop(columns=["stop_id", "route_id", "dwell_time", "event_time", "service_date", "trip_id", "headway"]).reindex(columns=["train_id", "event_time_sec"])
    runs = runs.merge(forest_hills_deps, left_on="train_id", right_on="train_id", how="inner", suffixes=("", "_fh_dep"))
    runs["train_timestamp"] = runs["event_time_sec"] - runs["event_time_sec_fh_dep"]
    runs = runs.drop(columns="event_time_sec_fh_dep")

    runs = runs.sort_values(["event_time_sec"])

    # all_trains_day = pd.concat(e)
    results[datetime.strptime(date, "%Y-%m-%d")] = runs, pd.concat(e)

# make dict a Dataframe to save to csv
# results = pd.concat(results)
# results.index.names = ["service_date", "cut_index"]
# results = results.reset_index(level="cut_index").drop(columns=["cut_index"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
runs = results.values().__iter__().__next__()[0]#.reindex(columns=["train_id", "service_date", "stop_id", "route_id", "dwell_time", "trip_id", "headway", "event_time", "event_time_sec"])

runs

,service_date,route_id,trip_id,stop_id,event_time,event_time_sec,train_id,dwell_time,headway,train_timestamp
0,2019-01-01,Orange,38939782,0,2019-01-01 06:00:26,21626,13,466.0,NaN,0
1,2019-01-01,Orange,38939782,1,2019-01-01 06:02:47,21767,13,42.0,NaN,141
2,2019-01-01,Orange,38939782,2,2019-01-01 06:04:28,21868,13,39.0,NaN,242
3,2019-01-01,Orange,38939782,3,2019-01-01 06:06:26,21986,13,48.0,NaN,360
4,2019-01-01,Orange,38939782,4,2019-01-01 06:08:06,22086,13,48.0,NaN,460
...,...,...,...,...,...,...,...,...,...,...
1667,2019-01-01,Orange,38939743,14,2019-01-02 01:24:35,91475,0,54.0,3522.0,3177
1668,2019-01-01,Orange,38939743,15,2019-01-02 01:26:43,91603,0,46.0,3462.0,3305
1669,2019-01-01,Orange,38939743,16,2019-01-02 01:28:43,91723,0,49.0,3445.0,3425
1670,2019-01-01,Orange,38939743,17,2019-01-02 01:30:21,91821,0,43.0,3419.0,3523
